# Лабораторная работа 2 -- "Читаем хабр, используем Наташу, изучаем именованные сущности)

## Задание 1.

Открываем википедию *(за что мне это)*, выбираем статью не менее 3000 слов, определяем именованные сущности, которые фигурируют в тексте, определяем, какие границы этих сущностей хотим задать.

In [31]:
import requests

# Хорошо, что так можно сделать
# Жаль, что мне это не понравилось

response = requests.get(
    'https://ru.wikipedia.org/w/api.php',
    params={
        'action': 'query',
        'format': 'json',
        'prop': 'extracts',
        'explaintext': True,
        'titles': 'Самоубийство',
    }
).json()

f = open('analyze.txt', 'w')
f.write(next(iter(response['query']['pages'].values()))['extract'])
f.close()

In [73]:
from natasha import Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger, NewsSyntaxParser, NewsNERTagger, PER, NamesExtractor, Doc 

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()

morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

text = open('analyze.txt').read()

doc = Doc(text)

# Рубрика "угадай, какую последовательность вызовов надо использовать, чтобы все это заработало"
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
for it in doc.tokens:
    it.lemmatize(morph_vocab)

doc.parse_syntax(syntax_parser)
doc.sents[1].syntax.print()

          ┌► Чрезвычайно  advmod
        ┌►└─ сложный      amod
        │ ┌► этический    amod
        └─└─ вопрос       nsubj
┌─┌─────└─┌─ представляет 
│ │     ┌─└► отнесение    obj
│ │     │ ┌► к            case
│ │ ┌───└►└─ самоубийству nmod
│ │ │ ┌────► (            punct
│ │ │ │ ┌──► либо         cc
│ │ │ │ │ ┌► к            case
│ │ └►└─└─└─ убийству     parataxis
│ │     └──► )            punct
│ └────────► эвтаназии    obl
└──────────► .            punct


In [ ]:
doc.tag_ner(ner_tagger)
doc.ner.print()
# аккуратно, тут будет много буков

In [105]:
for span in doc.spans:
    span.normalize(morph_vocab)

for span in doc.spans:
    if span.type == PER:
        span.extract_fact(names_extractor)

doc.spans[:15]

[DocSpan(stop=13, type='PER', text='Самоуби́йство', tokens=[...], normal='Самоуби́йство'),
 DocSpan(start=471, stop=477, type='LOC', text='Европы', tokens=[...], normal='Европа'),
 DocSpan(start=583, stop=626, type='ORG', text='Всемирной организации здравоохранения (ВОЗ)', tokens=[...], normal='Всемирная организация здравоохранения (ВОЗ)'),
 DocSpan(start=663, stop=668, type='LOC', text='Земли', tokens=[...], normal='Земля'),
 DocSpan(start=746, stop=749, type='ORG', text='ВОЗ', tokens=[...], normal='ВОЗ'),
 DocSpan(start=1030, stop=1047, type='LOC', text='Латинской Америке', tokens=[...], normal='Латинская Америка'),
 DocSpan(start=1119, stop=1130, type='LOC', text='Центральной', tokens=[...], normal='Центральная'),
 DocSpan(start=1133, stop=1148, type='LOC', text='Северной Европе', tokens=[...], normal='Северная Европа'),
 DocSpan(start=1150, stop=1166, type='LOC', text='Северной Америке', tokens=[...], normal='Северная Америка'),
 DocSpan(start=1168, stop=1186, type='LOC', text='Юго

In [107]:
doc.spans[2].tokens

[DocToken(start=583, stop=592, text='Всемирной', id='5_3', head_id='5_4', rel='amod', pos='ADJ', feats=<Gen,Pos,Fem,Sing>, lemma='всемирный'),
 DocToken(start=593, stop=604, text='организации', id='5_4', head_id='5_2', rel='nmod', pos='NOUN', feats=<Inan,Gen,Fem,Sing>, lemma='организация'),
 DocToken(start=605, stop=620, text='здравоохранения', id='5_5', head_id='5_4', rel='nmod', pos='NOUN', feats=<Inan,Gen,Neut,Sing>, lemma='здравоохранение'),
 DocToken(start=621, stop=622, text='(', id='5_6', head_id='5_7', rel='punct', pos='PUNCT', lemma='('),
 DocToken(start=622, stop=625, text='ВОЗ', id='5_7', head_id='5_5', rel='parataxis', pos='PROPN', feats=<Inan,Nom,Fem,Sing>, lemma='воз'),
 DocToken(start=625, stop=626, text=')', id='5_8', head_id='5_7', rel='punct', pos='PUNCT', lemma=')')]